In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
import tensorflow_probability as tfp
tfk = tf.keras
tf.keras.backend.set_floatx('float64')
tfd = tfp.distributions
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

In [3]:
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

#### The data

In [4]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv', usecols=['lat', 'long', 'id'])
kampala_devices.head()

,id,lat,long
0,930434,0.360209,32.610756
1,718028,0.307500,32.620600
2,912224,0.346460,32.703280
3,930426,0.365500,32.646800
4,930427,0.268900,32.588000


In [5]:
kampala_df = pd.read_csv('../data/kampala_data.csv', parse_dates=['timestamp'])
kampala_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
1,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
2,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
3,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
4,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26


In [6]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value', 'device_number']
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5,device_number
0,453031.0,0.356989,32.613888,10.5477,930434
1,453032.0,0.356989,32.613888,16.4250,930434
2,453033.0,0.356989,32.613888,17.7239,930434
3,453034.0,0.356989,32.613888,16.1533,930434
4,453035.0,0.356989,32.613888,18.0123,930434


In [7]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(35, 35, 34)

In [8]:
final_df = final_df.drop(['device_number'], axis=1)
final_df.head()

,time,latitude,longitude,pm2_5
0,453031.0,0.356989,32.613888,10.5477
1,453032.0,0.356989,32.613888,16.4250
2,453033.0,0.356989,32.613888,17.7239
3,453034.0,0.356989,32.613888,16.1533
4,453035.0,0.356989,32.613888,18.0123


#### Model training and validation

In [9]:
def bnn(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tfp.layers.DenseVariational(units=128, make_prior_fn=tfp.layers.default_multivariate_normal_fn, 
#                                 make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     tfp.layers.DenseVariational(units=64, make_prior_fn=tfp.layers.default_multivariate_normal_fn,
#                                make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     tfp.layers.DenseVariational(units=1, make_prior_fn=tfp.layers.default_multivariate_normal_fn,
#                                make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     ])
    
# #     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError())
#     model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
#     model.fit(X_train, y_train, batch_size=32, epochs=epochs, callbacks=None, validation_split=0.2)
    prior = tfd.Independent(tfd.Normal(loc=tf.zeros(1, dtype=tf.float64), scale=1.0), 
                            reinterpreted_batch_ndims=1)
    model = tfk.Sequential([
        tfk.layers.InputLayer(input_shape=(3,), name='input'),
        tfk.layers.Dropout(dropout, name='dropout1'),
        tfk.layers.Dense(10, activation='relu', name='dense_1'),
        tfk.layers.Dropout(dropout, name='dropout2'),
        tfk.layers.Dense(tfp.layers.MultivariateNormalTriL.params_size(1), activation=None, name='distribution_weights'),
        tfk.layers.Dropout(dropout, name='dropout3'),
        tfp.layers.MultivariateNormalTriL(1, activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/32), 
                                          name='output')], name='model')
    
#     model.compile(optimizer=optimizer, loss=neg_log_likelihood)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, callbacks=None, validation_split=0.2)
    return model

In [10]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
#     data_train =tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
#     data_train = data_train.batch(32).repeat(n_epochs)
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
#     data_test =tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
#     data_test = data_test.batch(1)
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
    
    model = bnn(X_train, y_train) #dropout=0.2
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [11]:
# rmse_list, mape_list = [], []
# for i in range(len(latitudes)):
#     rmse, mape = cross_validation(final_df, i)
#     rmse_list.append(rmse)
#     mape_list.append(mape)
#     print(f'{device_ids[i]} successful')

In [17]:
rmse_list, mape_list = [], []
# for i in range(len(latitudes[30:])):
lat_list = list(latitudes)
for lat in latitudes[30:]:
    i = lat_list.index(lat)
#     print(i)
    try:
        rmse, mape = cross_validation(final_df, i)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'Location {i} successful')
    except Exception as e:
        print(e)
        print(f'Location {i} failed')
#         break

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
mean_rmse, mean_mape

Epoch 1/1000
1167/1167 [==============================] - 3s 2ms/step - loss: 4396276819.7480 - val_loss: 1078.2604
Epoch 2/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 925.8651 - val_loss: 709.9106
Epoch 3/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 917.7274 - val_loss: 717.2851
Epoch 4/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 904.2922 - val_loss: 794.9878
Epoch 5/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 894.0153 - val_loss: 884.4697
Epoch 6/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 893.0514 - val_loss: 1003.0690
Epoch 7/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 883.5569 - val_loss: 980.6176
Epoch 8/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 881.4350 - val_loss: 705.6667
Epoch 9/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 881.4801 - val_loss: 941.1162
Epoch 10/1000
1167/

1167/1167 [==============================] - 1s 1ms/step - loss: 769.3038 - val_loss: 711.7737
Epoch 77/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 772.0488 - val_loss: 712.0543
Epoch 78/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 771.5764 - val_loss: 711.5303
Epoch 79/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 776.2624 - val_loss: 712.3571
Epoch 80/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 768.6885 - val_loss: 711.7300
Epoch 81/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 776.5660 - val_loss: 711.9130
Epoch 82/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 773.9841 - val_loss: 711.7595
Epoch 83/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 771.1488 - val_loss: 711.4228
Epoch 84/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 773.5909 - val_loss: 711.9086
Epoch 85/1000
1167/1167 [========

Epoch 151/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 780.0165 - val_loss: 713.2261
Epoch 152/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 770.0587 - val_loss: 712.5806
Epoch 153/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 777.6925 - val_loss: 712.8736
Epoch 154/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 771.6882 - val_loss: 712.7566
Epoch 155/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 776.7369 - val_loss: 712.8449
Epoch 156/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 770.4058 - val_loss: 712.5085
Epoch 157/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 775.3688 - val_loss: 712.6948
Epoch 158/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 776.7643 - val_loss: 712.7450
Epoch 159/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 771.6662 - val_loss: 712.4190
Epoch 160/

1167/1167 [==============================] - 1s 1ms/step - loss: 776.5391 - val_loss: 712.9310
Epoch 226/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 773.8377 - val_loss: 712.8386
Epoch 227/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 775.1727 - val_loss: 712.9188
Epoch 228/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 772.7265 - val_loss: 713.1030
Epoch 229/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 782.1219 - val_loss: 713.1578
Epoch 230/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 778.0851 - val_loss: 713.5180
Epoch 231/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 773.1890 - val_loss: 713.5061
Epoch 232/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 768.8293 - val_loss: 712.8343
Epoch 233/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 772.1949 - val_loss: 712.6178
Epoch 234/1000
1167/1167 

1167/1167 [==============================] - 1s 1ms/step - loss: 771.3446 - val_loss: 712.3064
Epoch 300/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 776.5294 - val_loss: 712.4535
Epoch 301/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 773.4956 - val_loss: 712.5175
Epoch 302/1000
1167/1167 [==============================] - 2s 1ms/step - loss: 770.6720 - val_loss: 712.1895
Epoch 303/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 778.0813 - val_loss: 712.7261
Epoch 304/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 770.4017 - val_loss: 712.7003
Epoch 305/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 773.3618 - val_loss: 712.5400
Epoch 306/1000
1167/1167 [==============================] - 2s 1ms/step - loss: 766.7115 - val_loss: 712.1738
Epoch 307/1000
1167/1167 [==============================] - 2s 1ms/step - loss: 776.0426 - val_loss: 712.5149
Epoch 308/1000
1167/1167 

...

1150/1150 [==============================] - 5s 4ms/step - loss: 769.6295 - val_loss: 493.6862
Epoch 708/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 777.8464 - val_loss: 494.2679
Epoch 709/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 776.5024 - val_loss: 494.4004
Epoch 710/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 775.2930 - val_loss: 494.2872
Epoch 711/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 776.5472 - val_loss: 494.2350
Epoch 712/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 765.4531 - val_loss: 494.1420
Epoch 713/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 770.8777 - val_loss: 493.7041
Epoch 714/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 767.6330 - val_loss: 493.6461
Epoch 715/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 782.4650 - val_loss: 493.9182
Epoch 716/1000
1150/1150 

1150/1150 [==============================] - 5s 4ms/step - loss: 775.1316 - val_loss: 494.0679
Epoch 782/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 766.6133 - val_loss: 494.1935
Epoch 783/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 776.6485 - val_loss: 494.1042
Epoch 784/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 773.8794 - val_loss: 493.8827
Epoch 785/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 780.9484 - val_loss: 494.1975
Epoch 786/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 771.8572 - val_loss: 494.1177
Epoch 787/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 782.6066 - val_loss: 494.3828
Epoch 788/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 776.9926 - val_loss: 494.2065
Epoch 789/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 779.6745 - val_loss: 494.3663
Epoch 790/1000
1150/1150 

1150/1150 [==============================] - 5s 4ms/step - loss: 772.2625 - val_loss: 494.1366
Epoch 856/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 777.0607 - val_loss: 494.3283
Epoch 857/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 770.9818 - val_loss: 494.0724
Epoch 858/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 774.9397 - val_loss: 494.0060
Epoch 859/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 774.5765 - val_loss: 494.3144
Epoch 860/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 776.7904 - val_loss: 494.2486
Epoch 861/1000
1150/1150 [==============================] - 5s 5ms/step - loss: 774.5057 - val_loss: 494.5385
Epoch 862/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 776.6807 - val_loss: 494.4458
Epoch 863/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 771.2082 - val_loss: 494.4423
Epoch 864/1000
1150/1150 

1150/1150 [==============================] - 5s 4ms/step - loss: 769.6606 - val_loss: 494.0288
Epoch 930/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 774.1470 - val_loss: 494.1412
Epoch 931/1000
1150/1150 [==============================] - 5s 5ms/step - loss: 769.9677 - val_loss: 493.8330
Epoch 932/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 769.0698 - val_loss: 493.4445
Epoch 933/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 772.3497 - val_loss: 493.3680
Epoch 934/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 776.7566 - val_loss: 493.8037
Epoch 935/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 768.2247 - val_loss: 493.5695
Epoch 936/1000
1150/1150 [==============================] - 5s 4ms/step - loss: 775.5493 - val_loss: 493.7978
Epoch 937/1000
1150/1150 [==============================] - 5s 5ms/step - loss: 777.3429 - val_loss: 493.6586
Epoch 938/1000
1150/1150 

(26.945651363690097, 0.44952596661153943)

In [20]:
rmse_list = [16.538255117721633,
 16.84502278200903,
 10.579277373251326,
 15.973294214171409,
 30.186567441901108,
 31.29482457193642,
 21.287963303115568,
 16.389732903985266,
 45.893530948824264,
 26.634679892001,
28.172270232972327,
 12.95149465152478,
 23.97742032799724,
 49.65400857462571,
 12.800111305829876,
 26.845253698709097,
 29.24009604522794,
 21.118996153578916,
 16.072073770015486,
 12.431960725817712,
 41.182668774076326,
 15.746585947399657,
 15.668914596456833,
 14.762447589141141,
 13.828138217481493,
 24.78328288787058,
 32.650258555703346,
 26.508558189828733,
 16.924253359434353,
 14.416310790435197,
18.781860232751455, 
33.174286662750525, 
18.64892457537078, 
37.177533983887635]
np.mean(rmse_list)

23.210025246994242

In [21]:
mape_list = [0.44524864301764233,
 0.6913634740032836,
 0.548728045229497,
 0.5722049871645247,
 0.4195071355555613,
 0.38869363678264834,
 0.5043249946909,
 0.45756940907714666,
 0.4583816526328937,
 0.38564707549165206,
0.4576904338440032,
 0.6198468692560332,
 0.5197654895434961,
 0.5061457910712694,
 0.9436976633805774,
 0.4843482444692084,
 0.36422056696558974,
 0.3917579212948447,
 0.5028241147814739,
 0.5434604825055787,
 0.4912645589343873,
 0.6621197676137234,
 0.5923610066664964,
 0.48741770062838635,
 0.4131870855868091,
 0.38438256356768913,
 0.4002403207464967,
 0.38665850775380817,
 0.5297587413701957,
 0.7142528910721765,
0.5017774025159843,
 0.4200460813178092,
 0.42301540789433156,
 0.45326497471803273]
np.mean(mape_list)

0.5019168717983573

In [15]:
mean_rmse = np.mean(rmse_list)          
mean_rmse

22.48675521970634

In [16]:
mean_mape = np.mean(mape_list)
mean_mape

0.5197700360526121